<a href="https://colab.research.google.com/github/Kenjiro1/Formacao_IA/blob/main/Projetos/Engenharia_de_Atributos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler


base  = pd.read_csv('https://raw.githubusercontent.com/Kenjiro1/Formacao_IA/refs/heads/main/dados/credit_simple.csv', sep=";")


In [ ]:
base.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [ ]:
y = base['CLASSE']
x = base.iloc[:, :-1]

In [ ]:
x.isnull().sum()

,0
SALDO_ATUAL,7
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,8
PROPOSITO,0


In [ ]:
mediana = x['SALDO_ATUAL'].median()
x['SALDO_ATUAL'].fillna(mediana, inplace=True)

/tmp/ipython-input-1628279515.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['SALDO_ATUAL'].fillna(mediana, inplace=True)


In [ ]:
x.isnull().sum()

,0
SALDO_ATUAL,0
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,8
PROPOSITO,0


In [ ]:
#Para categoricos como o estado civil vamos usar a moda
agrupado = x.groupby(['ESTADOCIVIL']).size()
agrupado

,0
ESTADOCIVIL,
fem div/cas,308
masculino casado/viuvo,92
masculino div/sep,50
masculino solteiro,542


In [ ]:
x['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
x.isnull().sum()

/tmp/ipython-input-2791302419.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  x['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)


,0
SALDO_ATUAL,0
RESIDENCIADESDE,0
IDADE,0
OUTROSPLANOSPGTO,0
DATA,0
ESTADOCIVIL,0
PROPOSITO,0


In [ ]:
#Detectar outliers
desvio = x['SALDO_ATUAL'].std()
desvio

685936688.9820064

In [ ]:
x.loc[x['SALDO_ATUAL']> 2 *desvio]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
127,2.541111e+09,2,30,nenhum,22/04/2020,masculino solteiro,obras
160,2.154441e+10,4,36,nenhum,22/05/2020,masculino solteiro,radio/tv


In [ ]:
x.loc[x['SALDO_ATUAL']> 2 *desvio]['SALDO_ATUAL'] = mediana
#Eliminamos os outliers


/tmp/ipython-input-614729578.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x.loc[x['SALDO_ATUAL']> 2 *desvio]['SALDO_ATUAL'] = mediana


#Tratamento com data biding


In [ ]:
agrupado = x.groupby(['PROPOSITO']).size()
agrupado

,0
PROPOSITO,
Eletrodomésticos,12
carro novo,234
carro usado,103
educação,50
mobilia/equipamento,181
negócios,97
obras,22
outros,12
qualificação,9


In [ ]:
x.loc[x['PROPOSITO']=='Eletrodomésticos','PROPOSITO'] = 'outros'
x.loc[x['PROPOSITO']=='qualificação','PROPOSITO'] = 'outros'
#agrupamentos essas categorias mais baixadas, isso eh data biding
agrupado = x.groupby(['PROPOSITO']).size()
agrupado

,0
PROPOSITO,
carro novo,234
carro usado,103
educação,50
mobilia/equipamento,181
negócios,97
obras,22
outros,33
radio/tv,280


In [ ]:
x['DATA'] #São considerados como categorias

,DATA
0,01/01/2019
1,01/01/2020
2,02/01/2020
3,02/01/2019
4,03/01/2018
...,...
995,29/06/2018
996,30/06/2018
997,03/07/2018
998,04/07/2019


In [ ]:
x['DATA'] = pd.to_datetime(x['DATA'], format ='%d/%m/%Y')
x['ANO'] = x['DATA'].dt.year
x['MES'] = x['DATA'].dt.month
x['DIASEMANA'] = x['DATA'].dt.day_name()
x.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,masculino solteiro,radio/tv,2019,1,Tuesday
1,5951.0,2,22,nenhum,2020-01-01,fem div/cas,radio/tv,2020,1,Wednesday
2,2096.0,3,49,nenhum,2020-01-02,masculino solteiro,educação,2020,1,Thursday
3,7882.0,4,45,nenhum,2019-01-02,masculino solteiro,mobilia/equipamento,2019,1,Wednesday
4,4870.0,4,53,nenhum,2018-01-03,masculino solteiro,carro novo,2018,1,Wednesday


In [ ]:
x['DIASEMANA']

,DIASEMANA
0,Tuesday
1,Wednesday
2,Thursday
3,Wednesday
4,Wednesday
...,...
995,Friday
996,Saturday
997,Tuesday
998,Thursday


In [ ]:
#Label encoder para tratar as colunas categoricas
x['ESTADOCIVIL'].unique()

array(['masculino solteiro', 'fem div/cas', 'masculino div/sep',
       'masculino casado/viuvo'], dtype=object)

In [ ]:
x['PROPOSITO'].unique()

array(['radio/tv', 'educação', 'mobilia/equipamento', 'carro novo',
       'carro usado', 'negócios', 'outros', 'obras'], dtype=object)

In [ ]:
x['DIASEMANA'].unique()

#One hot encoding gera colunas com a quantidade de categoria, por isso usaremos o label encoder nesse caso

array(['Tuesday', 'Wednesday', 'Thursday', 'Saturday', 'Sunday', 'Monday',
       'Friday'], dtype=object)

In [ ]:
LabelEncoder1 = LabelEncoder()
x['ESTADOCIVIL'] = LabelEncoder1.fit_transform(x['ESTADOCIVIL'])
x['PROPOSITO'] = LabelEncoder1.fit_transform(x['PROPOSITO'])
x['DIASEMANA'] = LabelEncoder1.fit_transform(x['DIASEMANA'])


In [ ]:
x.head() #Repare que todas essas colunas se tornaram como categorias ordinárias de


,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6


In [ ]:
outros= x['OUTROSPLANOSPGTO']
outros.unique()

array(['nenhum', 'banco', 'stores'], dtype=object)

In [ ]:
z = pd.get_dummies(x['OUTROSPLANOSPGTO'], prefix= 'OUTROS')
z

,OUTROS_banco,OUTROS_nenhum,OUTROS_stores
0,False,True,False
1,False,True,False
2,False,True,False
3,False,True,False
4,False,True,False
...,...,...,...
995,False,True,False
996,False,True,False
997,False,True,False
998,False,True,False


In [ ]:
x

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA
0,1169.0,4,67,nenhum,2019-01-01,3,7,2019,1,5
1,5951.0,2,22,nenhum,2020-01-01,0,7,2020,1,6
2,2096.0,3,49,nenhum,2020-01-02,3,2,2020,1,4
3,7882.0,4,45,nenhum,2019-01-02,3,3,2019,1,6
4,4870.0,4,53,nenhum,2018-01-03,3,0,2018,1,6
...,...,...,...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,2018-06-29,0,3,2018,6,0
996,3857.0,4,40,nenhum,2018-06-30,2,1,2018,6,2
997,804.0,4,38,nenhum,2018-07-03,3,7,2018,7,5
998,1845.0,4,23,nenhum,2019-07-04,3,7,2019,7,4


In [ ]:
sc = StandardScaler()
m = sc.fit_transform(x.iloc[:,0:3])
m

array([[-0.03513395,  1.04698668,  1.6392759 ],
       [-0.03512697, -0.76597727, -0.74024139],
       [-0.0351326 ,  0.14050471,  0.68746898],
       ...,
       [-0.03513448,  1.04698668,  0.1058092 ],
       [-0.03513296,  1.04698668, -0.68736323],
       [-0.03512898,  1.04698668, -0.47585058]])

In [ ]:
x = pd.concat([x,z,pd.DataFrame(m, columns = [['SALDO_ATUAL_N','RESIDENCIADESDE_N','IDADE_N']])],axis=1)

In [ ]:
x.drop(columns=['SALDO_ATUAL','RESIDENCIADESDE','IDADE','OUTROSPLANOSPGTO','DATA','OUTROS_banco'], inplace=True)

In [ ]:
x.head()

,ESTADOCIVIL,PROPOSITO,ANO,MES,DIASEMANA,OUTROS_nenhum,OUTROS_stores,"(SALDO_ATUAL_N,)","(RESIDENCIADESDE_N,)","(IDADE_N,)"
0,3,7,2019,1,5,True,False,-0.035134,1.046987,1.639276
1,0,7,2020,1,6,True,False,-0.035127,-0.765977,-0.740241
2,3,2,2020,1,4,True,False,-0.035133,0.140505,0.687469
3,3,3,2019,1,6,True,False,-0.035124,1.046987,0.475956
4,3,0,2018,1,6,True,False,-0.035129,1.046987,0.898982
